# Import Library

In [ ]:
import numpy as np
import pylab
import time
from kcss.filter.filter_design import FilterDesign
from kcss.filter.filter import Filter


# Specify Signal Parameters

In [ ]:
from kcss.modulators import constellation
# constellation and symbol map
const = np.array(constellation.CONSTELLATION["QPSK"])
symbol_map = np.arange(len(const))

# modulation parameters
sps = 5
rolloff = 0.5
n_taps = 101

# information to modulate
n_symbols = 50000
msg = np.random.randint(0, len(const), n_symbols)

# Python Version

In [ ]:
from kcss.modulators.pulse_shape_modulator import PulseShapeModulator

# design srrc filter
rrc = FilterDesign.design_rrc(sps, rolloff, n_taps)
pulse0 = Filter(rrc)

# create modulator
modulator = PulseShapeModulator(sps, const, symbol_map, pulse0)

# modulate signal
tic = time.time()
p_signal = modulator.modulate_symbols(msg)
print("Elapsed time %3.4f seconds"%(time.time()-tic))


# Java Version
TODO
* Calculate symbols by mapping to supplied constellation
* Apply filter on the upsampled symbols

In [ ]:
import jnius
JFilterObject = jnius.autoclass("jdsp.filters.FilterObject")
JFilterDesign = jnius.autoclass("jdsp.filters.FilterDesign")

# ----------------- design SRRC filter  ---------------------
pulse1 = JFilterDesign.designSRRC(n_taps, sps, 1.0, rolloff)

# create filter object and set numerator taps
pulse_fo = JFilterObject()
pulse_fo.setNumerator(pulse1)

# ------------  prepare input ------------------------
in_sig = np.zeros(len(msg) * int(sps), dtype=np.complex64)
in_sig[::int(sps)] = const[msg]
in_sig = in_sig.astype(np.complex64).view(np.float32)
in_sig = in_sig.tolist()

# -----------------  modulate  -----------------------
tic = time.time()
j_signal = pulse_fo.filterComplexInterleaved(in_sig)
print("Elapsed time %3.4f seconds"%(time.time()-tic))

# ---------------- convert to complex ------------------
j_signal = np.array(j_signal)
j_signal = j_signal[::2] + 1j* j_signal[1::2]



# Compare outputs

In [ ]:
# ----------------  plot generated signal  -----------------------
pylab.figure(figsize=[12,10])
pylab.subplot(231)
pylab.psd(p_signal)
pylab.subplot(232)
pylab.specgram(p_signal, Fs=3000.)
pylab.subplot(233)
pylab.plot(p_signal[:500].real)
pylab.plot(p_signal[:500].imag, 'r')
pylab.subplot(234)
pylab.psd(j_signal)
pylab.subplot(235)
pylab.specgram(j_signal, Fs=3000.)
pylab.subplot(236)
pylab.plot(j_signal[:500].real)
pylab.plot(j_signal[:500].imag, 'r')

pylab.show()
